EL PLANTEAMIENTO CON RESPECTO A LA PRIMERA VERSIÓN ES DIFERENTE:
A LA VISTA DE LOS ERRORES QUE SE PRODUCEN CON LA DESCARGA DEL TOTAL RETURN POR PARTES PRINCIPALMENTE DE LOS ÍNDICES DE RENTA FIJA, LO QUE SE HACE EN ESTE ALGORITMO ES DESCARGAR TODA LA SERIE TEMPORAL DE PRECIOS, ALINEARLAS TEMPORALMENTE Y POSTERIORMENTE CALCULAR LAS CARTERAS Y EL PERFORMANCE.
DE HECHO, ESTA VERSIÓN SE HA PLANTEADO DEBIDO A QUE ALGUNOS ÍNDICES NO ESTÁN REPRESENTADOS EN PORT <GO> Y LOS RESULTADOS QUE SE OBTIENEN NO SON REALES.

In [13]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
import bqviz as bqv
from scipy.optimize import minimize
from dateutil.relativedelta import relativedelta
import bqport
from bqport import PositionType

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

equity_idx = ['ACWI US Equity', 'EFA US Equity', 'XMEU GY Equity', 'VDEV LN Equity', 'IWVL LN Equity','IEMG US Equity','XRSU LN Equity']
fi_idx = ['FLOA LN Equity', 'IGIL LN Equity', 'CBU7 LN Equity']

etf_list = equity_idx + fi_idx

In [14]:
# INPUTS ###############

index_to_replicate = 'HFRIFOF Index'

start_date = datetime.datetime(2006,1,1)
end_date = datetime.datetime(2019,1,1)
curncy = 'USD'
rotation = 3 #months

In [15]:
#The fund inception of some ETFs it's too close to crrent days, so to make the analysis we will asume that these ETFs replicate exactly the underlying indexes.
#Due to this hypothesis, the time series are going to be analyzed with the underlying indexes.

etf_under_idx = d.fund_benchmark()

request = bql.Request(u.list(etf_list), {'UND_IDX': etf_under_idx})
res = bq.execute(request)
etf_under_idx_list = [undr +' Index' for undr in list(res[0].df()['UND_IDX'])]

#For the time period specified, the underlying indexes of VDEV LN Equity, ACIM US Equity and XRSU LN Equity there aren't "longer enough (they start later)
#We asume that the Tracking Error between these indexes and the original is quite low

for i in range(len(etf_under_idx_list)):
    if etf_under_idx_list[i] == 'AWNT04U Index':
        etf_under_idx_list[i] = 'FTAW01 Index'
    elif etf_under_idx_list[i] == 'M1WDIM Index':
        etf_under_idx_list[i] = 'MXWDIM Index'
    elif etf_under_idx_list[i] == 'RU20N30U Index':
        etf_under_idx_list[i] = 'RTY Index'

In [16]:
#Optimization functions
def f_obj(x):
    return sum((np.matmul(A, x) - y)**2)

def constraint_total_weight_ub(x):  #The sum of all weights must be 1
    return 1 - np.sum(x)

In [17]:
#Get the whole price series por all the indexes of the ETFs
download_start_date = start_date - relativedelta(months=12)
download_end_date = end_date + relativedelta(months=rotation)

price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
request = bql.Request(u.list(etf_under_idx_list), {'Price': price})
res = bq.execute(request)
price_series = res[0].df().reset_index()

#Pivot the price series to get the df in a way that the rows are dates and columns indexes prices
align_price_series = price_series.pivot(index='DATE', columns='ID', values='Price').reset_index().dropna().reset_index().drop(['index'], axis=1)

#Add to the price series df an auxiliar date 
year_list = pd.DatetimeIndex(align_price_series['DATE']).year.tolist()
month_list = pd.DatetimeIndex(align_price_series['DATE']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
align_price_series['aux_date'] = date_list  

#Get the price series for HFRIFOF Index (the returned data is in a monthly basis)
price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
request = bql.Request(index_to_replicate, {'Price': price})
res = bq.execute(request)
index_price_series = res[0].df().reset_index()

#Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
index_price_series = index_price_series.drop(['ID', 'CURRENCY'], axis=1)
index_price_series = index_price_series.rename(index=str, columns={'Price': index_to_replicate})
year_list = pd.DatetimeIndex(index_price_series['DATE']).year.tolist()
month_list = pd.DatetimeIndex(index_price_series['DATE']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
index_price_series['aux_date'] = date_list

grouped_price_series = pd.DataFrame(data = {})

list_aux_dates = align_price_series['aux_date'].unique().tolist()
for i_date in list_aux_dates:
    aux_mat = align_price_series.loc[align_price_series['aux_date'].isin([i_date]), align_price_series.columns != 'DATE']
    i_idx = aux_mat.index.tolist()
    aux_mat = aux_mat.loc[i_idx[-1]].to_frame().T
    
    if i_date == list_aux_dates[0]:
        grouped_price_series = aux_mat
        
    else:
        grouped_price_series = grouped_price_series.append(aux_mat, ignore_index=True)

grouped_price_series = pd.merge(index_price_series, grouped_price_series, how = 'left', on=['aux_date', 'aux_date'])

index_universe = [index_to_replicate]+etf_under_idx_list

aux_date = start_date
while end_date >= aux_date:
    print(aux_date)
    
    aux_date_st = aux_date - relativedelta(months=12)

    test_data = grouped_price_series.loc[(grouped_price_series['DATE'] >= aux_date_st) & (grouped_price_series['DATE'] <= aux_date)]
    test_data = test_data.loc[:,((grouped_price_series.columns != 'aux_date') & (grouped_price_series.columns != 'DATE')).tolist()]

    #Calculate the return
    test_data = test_data.pct_change().fillna(0)    
    
#     #Calculate the cumulative total return
#     test_data = (test_data.pct_change().fillna(0) + 1).cumprod() - 1    
    
    #Optimzation of series without UCITS constraints to see which is the optimal allocation

    A = []
    for i in range(len(test_data)):
        A += [test_data[etf_under_idx_list].iloc[i].values.tolist()]
    A = np.array(A)    

    y = test_data[index_to_replicate].tolist()
    y = np.array(y)

    guess=np.zeros(len(etf_under_idx_list))

    bnds = [(0,1) for i in range(len(etf_under_idx_list))]
    results = minimize(fun = f_obj, x0 = guess, method='SLSQP', bounds = bnds, options={'disp': True}, constraints= [{'type':'eq','fun' : constraint_total_weight_ub}])  
    
    if aux_date == start_date:
        port_df = pd.DataFrame({'date': aux_date,'security': etf_under_idx_list, 'weight': results.x})
    else :
        aux_port_df = pd.DataFrame({'date': aux_date,'security': etf_under_idx_list, 'weight': results.x})
        port_df = pd.concat([port_df, aux_port_df])

    aux_date = aux_date + relativedelta(months=rotation)
    
port_df

2006-01-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.000228022753584
            Iterations: 8
            Function evaluations: 96
            Gradient evaluations: 8
2006-04-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.000189520489374
            Iterations: 5
            Function evaluations: 60
            Gradient evaluations: 5
2006-07-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.000147681213218
            Iterations: 12
            Function evaluations: 144
            Gradient evaluations: 12
2006-10-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.000342671469296
            Iterations: 11
            Function evaluations: 132
            Gradient evaluations: 11
2007-01-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
       

date        security        weight
0  2006-01-01  NDUEACWF Index  8.692302e-02
1  2006-01-01  NDDUEAFE Index  8.815164e-02
2  2006-01-01   NDDUE15 Index  8.619597e-02
3  2006-01-01    FTAW01 Index  8.794582e-02
4  2006-01-01    M1WOEV Index  7.127946e-02
5  2006-01-01  MIMUEMRN Index  0.000000e+00
6  2006-01-01       RTY Index  7.014963e-02
7  2006-01-01  BFU5TRUU Index  1.704972e-01
8  2006-01-01    BCIW1A Index  1.694735e-01
9  2006-01-01  IDCOT3TR Index  1.693838e-01
0  2006-04-01  NDUEACWF Index  9.309823e-02
1  2006-04-01  NDDUEAFE Index  9.070732e-02
2  2006-04-01   NDDUE15 Index  9.423492e-02
3  2006-04-01    FTAW01 Index  9.493601e-02
4  2006-04-01    M1WOEV Index  7.773590e-02
5  2006-04-01  MIMUEMRN Index  0.000000e+00
6  2006-04-01       RTY Index  5.019494e-02
7  2006-04-01  BFU5TRUU Index  1.656580e-01
8  2006-04-01    BCIW1A Index  1.637896e-01
9  2006-04-01  IDCOT3TR Index  1.696451e-01
0  2006-07-01  NDUEACWF Index  9.957923e-02
1  2006-07-01  NDDUEAFE Index  5.956281e-02
2  2006-07-01   NDDUE15 Index  3.469425e-02
3  2006-07-01    FTAW01 Index  1.042460e-01
4  2006-07-01    M1WOEV Index  3.293383e-02
5  2006-07-01  MIMUEMRN Index  5.794963e-02
6  2006-07-01       RTY Index  5.168107e-02
7  2006-07-01  BFU5TRUU Index  2.459435e-01
8  2006-07-01    BCIW1A Index  9.076437e-02
9  2006-07-01  IDCOT3TR Index  2.226453e-01
..        ...             ...           ...
0  2018-07-01  NDUEACWF Index  1.103564e-01
1  2018-07-01  NDDUEAFE Index  8.510987e-18
2  2018-07-01   NDDUE15 Index  0.000000e+00
3  2018-07-01    FTAW01 Index  9.342653e-02
4  2018-07-01    M1WOEV Index  4.960225e-18
5  2018-07-01  MIMUEMRN Index  1.300669e-01
6  2018-07-01       RTY Index  6.889907e-02
7  2018-07-01  BFU5TRUU Index  2.859728e-01
8  2018-07-01    BCIW1A Index  3.859760e-17
9  2018-07-01  IDCOT3TR Index  3.112782e-01
0  2018-10-01  NDUEACWF Index  6.712196e-02
1  2018-10-01  NDDUEAFE Index  4.010847e-02
2  2018-10-01   NDDUE15 Index  2.276825e-18
3  2018-10-01    FTAW01 Index  6.414938e-02
4  2018-10-01    M1WOEV Index  1.810618e-17
5  2018-10-01  MIMUEMRN Index  1.253928e-01
6  2018-10-01       RTY Index  8.276237e-02
7  2018-10-01  BFU5TRUU Index  3.145038e-01
8  2018-10-01    BCIW1A Index  3.968180e-17
9  2018-10-01  IDCOT3TR Index  3.059613e-01
0  2019-01-01  NDUEACWF Index  5.923232e-02
1  2019-01-01  NDDUEAFE Index  4.875475e-02
2  2019-01-01   NDDUE15 Index  2.496456e-02
3  2019-01-01    FTAW01 Index  5.486529e-02
4  2019-01-01    M1WOEV Index  3.655140e-02
5  2019-01-01  MIMUEMRN Index  3.469447e-18
6  2019-01-01       RTY Index  8.201808e-02
7  2019-01-01  BFU5TRUU Index  2.372183e-01
8  2019-01-01    BCIW1A Index  2.071902e-01
9  2019-01-01  IDCOT3TR Index  2.492051e-01

[530 rows x 3 columns]

In [19]:
#Securities don't follow the same order in different time periods
port_df = port_df.sort_values(by=['date', 'security'])

#Add to the portfolio df an auxiliar date 
year_list = pd.DatetimeIndex(port_df['date']).year.tolist()
month_list = pd.DatetimeIndex(port_df['date']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
port_df['aux_date'] = date_list  

#Get the price and consecuently the number of shares for each moment of the rebalancing date
list_reb_dates = port_df['aux_date'].unique().tolist()

price_0_list = []
for i_date in list_reb_dates:
    aux_port_df = port_df.loc[port_df['aux_date'].isin([i_date])]
    price_0_list += grouped_price_series.loc[grouped_price_series['aux_date'].isin([i_date])][aux_port_df['security'].tolist()].values[0].tolist()
port_df['Price'] = price_0_list
port_df['N_Assets_type'] = port_df['weight']/port_df['Price']

#For the first date the calculated number of shares it's OK, but for the rest of the dates it must be take into account the value of the portfolio in that moment
aux_df_real_n_assets = port_df.loc[port_df['aux_date'].isin([list_reb_dates[0]])][['aux_date', 'N_Assets_type']]
    
for i_prev_date,i_date in zip(list_reb_dates[:-1], list_reb_dates[1:]):
    n_assets = np.array(aux_df_real_n_assets.loc[aux_df_real_n_assets['aux_date'].isin([i_prev_date])]['N_Assets_type'].tolist())
    cur_price = np.array(port_df.loc[port_df['aux_date'].isin([i_date])]['Price'].tolist())
    val = sum(n_assets*cur_price)
    cur_n_assets = val*np.array(port_df.loc[port_df['aux_date'].isin([i_date])]['weight'].tolist())/cur_price
    
    aux_df_real_n_assets = aux_df_real_n_assets.append(pd.DataFrame(data={'aux_date': [i_date]*len(cur_n_assets), 'N_Assets_type': cur_n_assets}), ignore_index = True)

port_df['Real_N_assets'] = aux_df_real_n_assets['N_Assets_type'].tolist()

#Put the portfolio df with the number of shares in each period of time in the same format as the price series df
port_df_pivot = port_df.pivot(index = 'aux_date', columns = 'security', values = 'Real_N_assets').reset_index()
n_assets_type_table = pd.merge(align_price_series['aux_date'].to_frame(), port_df_pivot, how='left', on=['aux_date', 'aux_date']).fillna(method='ffill')

#Compute the portfolio value
port_val = align_price_series.copy()
for asset in etf_under_idx_list:
    port_val[asset] = port_val[asset] * n_assets_type_table[asset]
    
port_val['Total_Abs'] = port_val[etf_under_idx_list].sum(axis=1)

#Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
def_series = index_price_series.copy()

aligned_port_vals = []
for i_date in def_series['aux_date'].tolist():
    aligned_port_vals += [port_val.loc[port_val['aux_date'].isin([i_date])]['Total_Abs'].values[-1]]

#Calculate the cumulated total return of both value series
def_series['Port'] = aligned_port_vals
def_series = def_series.dropna(subset = ['Port'])
def_series[['Port', index_to_replicate]] = (def_series[['Port', index_to_replicate]].pct_change().fillna(0) + 1).cumprod() - 1

#Sort Columns
def_series = def_series[['DATE', 'Port', index_to_replicate, 'aux_date']]

#Plot the result
plot_def_series = def_series.drop(['aux_date'], axis=1).set_index('DATE')
result = bqv.ComparisonPlot(plot_def_series)

# Display the plot
result.show()